# CRIM Intervals:  Cadences

### Reminders:

#### Import Music Files

* If you are exploring pieces from CRIM, importing simply involves providing the CRIM URL of the MEI file:  
    * **`piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')`**

* But you can also use the Notebook with any MEI, MusicXML, or MIDI file of your own. You can easily do this when you run the Notebooks on Jupyter Hub, you will also find a folder called **`Music_Files`**.  Upload the file here, then provide the path to that file: 
    * **`piece = importScore('Music_Files/My_File_Name.mei')`**.  

#### Save outputs as CSV or Excel

* The Jupyter Hub version of these Notebooks also provides a folder called **`saved_csv`**.  You can save **csv** files of any data frame there with this command: 
    * **`notebook_data_frame_name.to_csv('saved_csv/your_file_title.csv')`**.
* If you prefer **Excel** documents (which are better for anything with a complex set of columns or hierarhical index), use **ExcelWriter**.  In the following code, you will need to provide these commands:
    * **`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')`**
* Now convert your dataframe to Excel
    * **`frame_name.to_excel(writer, sheet_name='Sheet1')`**
* And finally save the new file to the folder here in the Notebook:
    * **`writer.save()`**

Put the following code to a new cell and update the frame_name and file_name:

`writer = pd.ExcelWriter('saved_csv/file_name.xlsx', engine='xlsxwriter')` <br>
`frame_name.to_excel(writer, sheet_name='Sheet1')` <br>
`writer.save()` <br>


## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [35]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## B. Importing a Piece

### B.1 Import a Piece and Check Title

In [36]:
# Select a prefix:

prefix = 'Music_Files/'
# prefix = 'https://crimproject.org/mei/'

# Add your filename here

mei_file = 'CRIM_Model_0038.mei'

url = prefix + mei_file

piece = importScore(url)

print(piece.metadata)


Memoized piece detected.
{'title': 'Ultimi miei sospiri', 'composer': 'Philippe Verdelot'}


## C. Find Cadences with Modular Analysis

Alex Morgan has built a powerful tool that identifies cadences according to the combinations of two-voice **modules** that describe the typical contrapuntal motion between the various **cadential voice functions** (**CVF**) heard in Renaissance polyphony:  **cantizans and tenorizans**, **cantizans and bassiszans**, etc.

The tool uses modular analysis to identify **conjunctions** of these pairs in order to predict cadences of various kinds. But there are many combinations, especially once we consider that voices functions (or roles) can be **displaced** (as when the tenorizans role appears in the Superius part and the cantizans appears in the Tenor part), or through **irregular** motion, and even **interrupted**, as when a voice is suddenly silent. 

You can in fact check all of the cadential voice functions (CVFs) for a given piece below.  But Alex's system also conveniently **labels** the cadences according to **type**, **tone**, **evaded** and also provides information about the relative place within the piece, the adjacent cadences, and many other features, too.

Note:  **Measure** and **Beat** columns are in the body of the table, not at the Index.

**Column Headings Explained**:

* The **Low** and **Tone** columns give the pitches of the **lowest sounding pitch (in any voice) at the perfection**, and the **goal tone of the cantizans** (or altizans if there is no cantizans) respectively.

* **RelLow** is the lowest pitch of each cadence shown as an interval measured against the last pitch in the **Low** column. Likewise, **RelTone** is the cadential tone shown as an interval measured against the last pitch in the **Tone** column.

* The **SinceLast** and **ToNext** columns are the time in quarter notes since the last or to the next cadence.

* The **Progress** column is a relative indication of position in the piece.  **0** is the beginning of the piece; **1.0** is the end of the piece.



Read more via the documentation: **`print(piece.cadences.__doc__)`**

View the **Cadential Voice Function** and **Cadence Label** tables here:  https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/.  These can easily be updated with revised or new cadence types.

### C.1 Classify the Cadences

In [39]:
cads = piece.cadences()
cads

,CadType,Evaded,LeadingTones,Low,RelLow,Tone,RelTone,TSig,Measure,Beat,Progress,SinceLast,ToNext
40.0,Phrygian,False,1.0,D3,P5,A,P5,4/2,6,1.0,0.070423,40.0,36.0
76.0,Clausula Vera,True,-1.0,D3,P5,NaN,NaN,4/2,10,3.0,0.133803,36.0,20.0
96.0,Authentic,False,1.0,D3,P5,D,P1,4/2,13,1.0,0.169014,20.0,12.0
108.0,Clausula Vera,True,-1.0,G2,P1,NaN,NaN,4/2,14,3.0,0.190141,12.0,12.0
120.0,Clausula Vera,False,1.0,G2,P1,G,P4,4/2,16,1.0,0.211268,12.0,40.0
160.0,Phrygian,False,1.0,D3,P5,A,P5,4/2,21,1.0,0.281690,40.0,36.0
196.0,Clausula Vera,True,-1.0,D3,P5,NaN,NaN,4/2,25,3.0,0.345070,36.0,20.0
216.0,Authentic,False,1.0,D3,P5,D,P1,4/2,28,1.0,0.380282,20.0,12.0
228.0,Clausula Vera,True,-1.0,G2,P1,NaN,NaN,4/2,29,3.0,0.401408,12.0,12.0
240.0,Clausula Vera,False,1.0,G2,P1,G,P4,4/2,31,1.0,0.422535,12.0,8.0


In [40]:
# Reorganize the Columns:
col_list = ['Measure', 'Beat', 'CadType', 'Tone','Evaded','LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext']
cads = cads[col_list]
cads

,Measure,Beat,CadType,Tone,Evaded,LeadingTones,Low,RelLow,RelTone,Progress,SinceLast,ToNext
40.0,6,1.0,Phrygian,A,False,1.0,D3,P5,P5,0.070423,40.0,36.0
76.0,10,3.0,Clausula Vera,NaN,True,-1.0,D3,P5,NaN,0.133803,36.0,20.0
96.0,13,1.0,Authentic,D,False,1.0,D3,P5,P1,0.169014,20.0,12.0
108.0,14,3.0,Clausula Vera,NaN,True,-1.0,G2,P1,NaN,0.190141,12.0,12.0
120.0,16,1.0,Clausula Vera,G,False,1.0,G2,P1,P4,0.211268,12.0,40.0
160.0,21,1.0,Phrygian,A,False,1.0,D3,P5,P5,0.281690,40.0,36.0
196.0,25,3.0,Clausula Vera,NaN,True,-1.0,D3,P5,NaN,0.345070,36.0,20.0
216.0,28,1.0,Authentic,D,False,1.0,D3,P5,P1,0.380282,20.0,12.0
228.0,29,3.0,Clausula Vera,NaN,True,-1.0,G2,P1,NaN,0.401408,12.0,12.0
240.0,31,1.0,Clausula Vera,G,False,1.0,G2,P1,P4,0.422535,12.0,8.0


In [43]:
cads['Tone'].value_counts().to_frame()

,Tone
D,6
G,6
A,2


### C.2  Check Voice Functions for all Cadences of a Given Piece

**`piece.cvfs()`** will return a table
of the **cadential voice functions** (CVFs). Each CVF is
represented with a single-character label with the meanings as follows:

* `C`: **cantizans motion up a step** (can also be ornamented e.g. Landini)
* `T`: **tenorizans motion down a step** (can be ornamented with anticipations)
* `B`: **bassizans motion up a fourth or down a fifth**
* `A`: **altizans motion, similar to cantizans**, but cadences to a fifth
above a tenorizans instead of an octave  <br><br>
* `L`: **leaping contratenor** motion up an octave at the perfection
* `P`: **plagal bassizans** motion up a fifth or down a fourth <br><br>
* `c`: **evaded cantizans** when it moves to an unexpected note at the perfection
* `t`: **evaded tenorizans** when it goes up by step at the perfection
* `b`: **evaded bassizans** when it goes up by step at the perfection
* `u`: **evaded bassizans** when it goes down by third at the perfection
(there are no evaded labels for the altizans, plagal bassizans leaping
contratenor CVFs) <br><br>
* `x`: evaded **bassizans motion where the voice drops out** at the perfection
* `y`: evaded **cantizans motion where the voice drops out** at the perfection
* `z`: evaded **tenorizans motion where the voice drops out** at the perfection


View the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

In [10]:
cvf_list = piece.cvfs()
piece.detailIndex(cvf_list, offset=True).fillna('-')


,,,Cantus,Altus,Quintus,Tenor,Sextus,Bassus
Measure,Beat,Offset,,,,,,
6,1.0,40.0,C,-,-,T,-,-
10,3.0,76.0,T,-,-,-,c,-
13,1.0,96.0,T,C,-,-,-,B
14,3.0,108.0,-,-,T,-,-,c
16,1.0,120.0,-,-,-,C,-,T
21,1.0,160.0,C,-,-,T,-,-
25,3.0,196.0,T,-,-,-,c,-
28,1.0,216.0,T,C,-,-,-,B
29,3.0,228.0,-,-,T,-,-,c


### C.3.  Check for Missed Cadences (One or More Pieces)


* The resulting table lists the **Cadential Voice Functions** (CVF's) for each of the probable cadences that the Cadence Classifier was **unable to categorize** according to the **Cadential Voice Function** and **Cadence Label** tables here: https://github.com/HCDigitalScholarship/intervals/tree/main/intervals/data/cadences/. These can easily be updated with revised or new cadence types.

* See below for additional instructions.

In [32]:

corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei']) # add as many pieces as you want here, each in single quotations, separated by commas
cad_dfs = corpus.batch(ImportedPiece.cadences, metadata=False)
cvf_dfs = corpus.batch(ImportedPiece.cadences)
missed = []
for i, cad in enumerate(cad_dfs):
    df = cvf_dfs[i].loc[cad.CadType.isnull(), :]
    df = df[df.columns[range(-2, len(df.columns) -2)]]
    df.columns = range(len(df.columns))
    missed.append(df)
result = pd.concat(missed)
rm = piece.detailIndex(result, offset=True)
result_offset_list = rm.index.get_level_values('Offset')



Memoized piece detected.


### C.4.  Check the Harmonic Modules of Missing Cadences

* For a given piece (since this tool works for one composition at a time), you can check the **harmonic modules** (for example `7_Held, 6_-2, 8` for a cantizans-tenorizans pair) at any given point in your piece
    * **Import the piece** (just ONE at a time)
    * Run the **"missed cadence finder"** (Section D) above.  You must run this with just ONE piece!
* The default **modular ngram** is 3 events long.  Adjust **`n`** as needed for longer modules
* The default **interval type** is diatonic ('d').  Adjust **`kind`** as needed for chromatic ('c')

Use the results to report the new cadential voice functions and label, as explained above.

In [31]:

ngs = piece.ngrams(n=3)
ngs_filtered = ngs.reindex(index = result_offset_list)
ngs_filtered.dropna(how='all').fillna('-')



,Bassus_Sextus,Bassus_Tenor,Bassus_Quintus,Bassus_Altus,Bassus_Cantus,Sextus_Tenor,Sextus_Quintus,Sextus_Altus,Sextus_Cantus,Tenor_Quintus,Tenor_Altus,Tenor_Cantus,Quintus_Altus,Quintus_Cantus,Altus_Cantus
Offset,,,,,,,,,,,,,,,
84.0,-,-,-,-,-,-,-,"3_Held, 5_-2, 6","10_-2, 11_Held, 10",-,-,-,-,-,"8_3, 6_2, 5"
116.0,-,"7_Held, 6_-2, 8","10_-2, 11_Held, 10",-,-,"5_Held, 4_2, 4","8_2, 7_Held, 6",-,-,"4_-2, 5_2, 4",-,-,-,-,-
